In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from getpass import getpass
from urllib.parse import quote

import pandas as pd
from SPARQLWrapper import JSON, SPARQLWrapper
from tqdm.notebook import tqdm

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import s3, iiif

try:
    S3_CLIENT_ID
except NameError:
    S3_CLIENT_ID = getpass("Please input s3 client id")

try:
    S3_CLIENT_SECRET
except NameError:
    S3_CLIENT_SECRET = getpass("Please input s3 client secret")


tqdm.pandas()
def identifier_to_base_url(identifier):
    return "https://media.performing-arts.ch/iiif/manifest/" + quote(identifier)

def get_full_image_url(image_basename):
    return f"https://media.performing-arts.ch/iiif/3/image%2F{image_basename}"

In [2]:
sparql = SPARQLWrapper("https://www.performing-arts.ch/sparql")

sparql.setQuery(
    """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX spav: <http://vocab.performing-arts.ch/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
SELECT ?record ?label ?identifier ?copyright ?creator
WHERE {
  ?record a rico:Record; rico:isOrWasIncludedIn+ <http://data.performing-arts.ch/r/caead728-064b-4fed-8d81-a8ab2c328f97>;
                         rico:hasOrHadIdentifier ?Cote;
                         schema:copyrightNotice ?copyright;
                         rico:name ?label;
                         rico:hasCreator ?CreatorNode.
  ?CreatorNode rdfs:label ?creator.
  ?Cote rdf:value ?identifier; crm:P2_has_type spav:iduni.
} ORDER BY ?CoteValue
"""
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

metadata= pd.DataFrame.from_dict(
    [{k: v["value"] for k, v in x.items()} for x in results["results"]["bindings"]]
)

In [3]:
if not metadata['record'].is_unique:
    print("SPARQL query returned duplicated records")

SPARQL query returned duplicated records


In [4]:
metadata.loc[metadata.duplicated("record", keep=False)].sort_values("record")

,record,copyright,label,identifier,creator
220,http://data.performing-arts.ch/r/04fd3f9b-334a...,"Sandra Sibiglia, Gunter Elmer",Hier sind sie richtig (1971). Bild 1,1552-FO-332-1,Gunter Elmer
264,http://data.performing-arts.ch/r/04fd3f9b-334a...,"Sandra Sibiglia, Gunter Elmer",Hier sind sie richtig (1971). Bild 1,1552-FO-332-1,Sandra Sibiglia
124,http://data.performing-arts.ch/r/19d173f7-5866...,"Sandra Sibiglia, Gunter Elmer",Die Räuber (1972). Bild 1,1552-FO-351-1,Gunter Elmer
167,http://data.performing-arts.ch/r/19d173f7-5866...,"Sandra Sibiglia, Gunter Elmer",Die Räuber (1972). Bild 1,1552-FO-351-1,Sandra Sibiglia
178,http://data.performing-arts.ch/r/68838e64-9762...,"Sandra Sibiglia, Gunter Elmer",Die chinesische Mauer (1973). Bild 1,1552-FO-371-1,Gunter Elmer
218,http://data.performing-arts.ch/r/68838e64-9762...,"Sandra Sibiglia, Gunter Elmer",Die chinesische Mauer (1973). Bild 1,1552-FO-371-1,Sandra Sibiglia
365,http://data.performing-arts.ch/r/9cef1f74-0774...,"Sandra Sibiglia, Willi Gasché",Die Zauberflöte (1971). Bild 1,1552-FO-336-1,Sandra Sibiglia
369,http://data.performing-arts.ch/r/9cef1f74-0774...,"Sandra Sibiglia, Willi Gasché",Die Zauberflöte (1971). Bild 1,1552-FO-336-1,Willi Gasché
320,http://data.performing-arts.ch/r/ae42a4aa-de14...,"Sandra Sibiglia, Gunter Elmer",Can-Can (1972). Bild 1,1552-FO-360-1,Gunter Elmer
364,http://data.performing-arts.ch/r/ae42a4aa-de14...,"Sandra Sibiglia, Gunter Elmer",Can-Can (1972). Bild 1,1552-FO-360-1,Sandra Sibiglia


In [5]:
metadata = metadata.groupby("record").agg(lambda x: list(set(x)) if len(set(x)) > 1 else x.iat[0]).reset_index()

In [6]:
with open('./file_list', "r") as infile:
    original_files = pd.DataFrame(pd.Series(infile.readlines()).rename("path").str.strip("\n"))
original_files['filename'] = original_files["path"].apply(lambda path: os.path.basename(path))
original_files['basename'] = original_files['filename'].str.replace(".tif$", "", regex=True)
original_files['identifier'] = original_files['basename'].str.replace('-PC$', "", regex=True)

In [7]:
len(original_files)

788

Files in metadata and not if original files:

In [8]:
metadata.loc[~metadata['identifier'].isin(original_files['identifier'])]

,record,copyright,label,identifier,creator
50,http://data.performing-arts.ch/r/108df43b-bf29...,Michael von Graffenried,Das Leben ist Traum (1987). Bild 1,1552-FO-635-1,Michael von Graffenried
675,http://data.performing-arts.ch/r/ddecebc2-2c52...,Michael von Graffenried,I Pagliacci (Der Bajazzo). Oper in zwei Akten ...,1552-FO-545-1-1,Michael von Graffenried


Files in original files and not in metadata:

In [9]:
original_files.loc[~original_files['identifier'].isin(metadata['identifier'])]

,path,filename,basename,identifier
427,./HiRes-16bit-PC/1552-FO-545-1-PC.tif,1552-FO-545-1-PC.tif,1552-FO-545-1-PC,1552-FO-545-1
511,./HiRes-16bit-PC/1552-FO-635- 1-PC.tif,1552-FO-635- 1-PC.tif,1552-FO-635- 1-PC,1552-FO-635- 1


Let's say they match.

In [10]:
original_files.loc[original_files['identifier'] == '1552-FO-545-1', 'identifier'] = '1552-FO-545-1-1'
original_files.loc[original_files['identifier'] == '1552-FO-635- 1', 'identifier'] = '1552-FO-635-1'

Add filename to metadata.

In [11]:
metadata['iiif_url'] = metadata['identifier'].apply(get_full_image_url) + "-PC"

In [12]:
def metadata_to_iiif(row):
    manifest_base_url = identifier_to_base_url(row["identifier"])
    rights = "http://rightsstatements.org/vocab/InC/1.0/"
    if "CC BY-SA 4.0" in row["copyright"]:
        rights = "http://creativecommons.org/licenses/by-sa/4.0/"
    manifest_base_metadata = iiif.get_manifest_base_metadata(
        row["copyright"], rights=rights
    )
    return iiif.create_manifest_from_iiif_images(
        manifest_base_url=manifest_base_url,
        iiif_images=[
            iiif.IIIFImageItem(
                row["iiif_url"], show_rendering=False
            )
        ],
        base_metadata=manifest_base_metadata,
        label=row["label"],
        summary=row["label"],
        sapa_resource=row["record"],
        identifier=row["identifier"],
        description="Aufnahmen der Produktionen des Stadttheater Bern",
        creator=row["creator"],
    )

In [13]:
manifests = []
for _, row in tqdm(metadata.iterrows(), total=len(metadata)):
    manifest = metadata_to_iiif(row)
    manifests.append(manifest)

  0%|          | 0/788 [00:00<?, ?it/s]

In [14]:
manifests[1]['id'].split('/')[-1]

'1552-FO-943-1.json'

In [15]:
client = s3.S3Client(S3_CLIENT_ID, S3_CLIENT_SECRET)

for manifest in tqdm(manifests):
    client.upload_as_json(manifest, "manifests/" + manifest['id'].split('/')[-1])

  0%|          | 0/788 [00:00<?, ?it/s]

In [ ]:
from rdflib import XSD, Graph, Literal, URIRef
from rdflib.namespace import RDF, RDFS
import requests
from urllib.parse import quote as url_encode

from SPARQLWrapper import JSON, SPARQLWrapper

try: SPARQL_USERNAME
except NameError: SPARQL_USERNAME = input("Please input SPARQL username")

try: SPARQL_PASSWORD
except NameError: SPARQL_PASSWORD = input("Please input SPARQL password")


def get_graph_by_uri(uri):
    sparql = SPARQLWrapper("https://www.performing-arts.ch/sparql")
    sparql.setQuery(
        f"""
SELECT ?graph
WHERE {{
  GRAPH ?graph {{ 
    <{uri}> a ?type .
  }}
}}
"""
    )
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()["results"]["bindings"]
    assert len(results) > 0, "Found no matching graph"
    results = pd.DataFrame.from_dict(
        [{k: v["value"] for k, v in x.items()} for x in results]
    )
    assert len(results['graph'].unique()) == 1, f"Found more than one matching graph, found {len(results['graph'].unique())}"
    return results['graph'].iloc[0]

base_ldp_url = 'https://www.performing-arts.ch/rdf-graph-store?graph='
def get_graph(graph_uri, ldp_url=base_ldp_url):
    graph_request = requests.get(ldp_url + url_encode(graph_uri, safe=''), auth=(SPARQL_USERNAME, SPARQL_PASSWORD), headers={'Accept': 'application/rdf+xml; charset=UTF-8'})
    graph_request.raise_for_status()
    graph = Graph()
    graph.parse(data=graph_request.text, format='xml')
    return graph

def update_graph(graph, graph_uri, ldp_url=base_ldp_url):
    serialized_graph = graph.serialize(format='pretty-xml', encoding='utf-8')
    graph_update_request = requests.put(ldp_url + url_encode(graph_uri, safe=''), data=serialized_graph, auth=(SPARQL_USERNAME, SPARQL_PASSWORD), headers={'Content-Type': 'application/rdf+xml; charset=UTF-8'})
    graph_update_request.raise_for_status()



In [ ]:
def add_iiif_manifest(row):
    uri = URIRef(row["record"])
    graph_uri = get_graph_by_uri(uri)
    graph = get_graph(graph_uri)
    iiif_uri = URIRef(
        f"https://media.performing-arts.ch/iiif/manifest/{row['identifier']}.json"
    )
    
    graph.add(
        (
            iiif_uri,
            RDF.type,
            URIRef("https://www.ica.org/standards/RiC/ontology#Instantiation"),
        )
    )
    
    graph.add(
        (
            iiif_uri,
            URIRef("https://www.ica.org/standards/RiC/ontology#hasRepresentationType"),
            URIRef("http://vocab.performing-arts.ch/rtvs"),
        )
    )
    
    graph.add(
        (
            iiif_uri,
            URIRef("https://www.ica.org/standards/RiC/ontology#hasCarrierType"),
            URIRef("http://vocab.performing-arts.ch/fvman"),
        )
    )
    
    graph.add(
        (
            uri,
            URIRef("https://www.ica.org/standards/RiC/ontology#hasInstantiation"),
            iiif_uri,
        )
    )
    
    update_graph(graph, graph_uri)

In [ ]:
added_idx = set()

In [ ]:
import time

for idx, row in tqdm(metadata.iterrows(), total=len(metadata)):
    if idx not in added_idx:
        add_iiif_manifest(row)
        added_idx.add(idx)
        # print(f"{idx} has been added")
        time.sleep(0.2)

  0%|          | 0/788 [00:00<?, ?it/s]

100%|██████████| 788/788 [00:25<00:00, 30.61it/s] 


In [ ]:
for idx, row in tqdm(metadata.iterrows(), total=len(metadata)):
    print(idx)


100%|██████████| 788/788 [00:00<00:00, 30723.22it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
metadata.iloc[0]['record']

'http://data.performing-arts.ch/r/0038b81e-f1ac-407b-8391-51b2eaf83504'